# Smart Logistic System in Amazon 

### Note:  

1. This is a group-based project, you are only allowed to discuss with your team members.

2. You are not allowed to use any packages to read or write csv files.
3. You are not allowed to use dynamic programming for this problem.

In [1]:
import numpy as np   # Numpy package would be the only package you could use for this project.
                     # Please do not use pandas or other packages.

## Problem1: The Estimated Value for Each Warehouse

In this problem 1, calculating the estimated total value and accumulated weight of each of 300 warehouses will be helpful for the later decision of cross-warehouse-transshipment solution in problem 2. 

### Step 1: Estimate the Maximized Total Value and Accumulated Weight of One Warehouse

Read the dataset from  "Products.csv" first.

In [2]:
file_name='Products.csv'

# Write your codes here
items = open(file_name, "r")
products = items.readlines()
products 


# Strip unnecessary punctuations
products = [n.strip('\n').split(',') for n in products if n != ',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n']

# Convert strings to float
products = [[float(num) for num in n] for n in products]

Now let's get the estimated value and weight for one warehouse.You need to write a function "get_max_value" which could do the following 4 things:
1. Calculate the Value_Weight Ratio for each product. 
2. Sort the calculated Value_Weight Ratios in descending order. 
3. Select the products from the top of the descending order to the bottom while looking at the accumulated         weight of products to make sure the accumulated weight does not exceed the warehouse weight capacity.
4. Calculate the  corresponding estimated value for this warehouse after finishing product selection.

Remember, take every product into consideration; otherwise, you may have the risk of missing products. You want to put as many product with highest Value_Weight Ratio as possible before reaching the weight capacity for this warehouse.

In [3]:
def get_max_value(products, i):  # i means ith warehouse (i = 0 ~ 299)
    
    #write your codes here
    
    # Define 'weights', 'values' from 'products'
    weights = products[i*2]  # 0th, 2nd, ..., 598th lines 
    values = products[i*2 + 1]  # 1st, 3rd, ..., 599th lines
    
    # Define 'vw_ratios' (Value-Weight Ratios)
    vw_ratios = np.array(values) / np.array(weights)  # Convert to numpy array for calculation
    vw_ratios = vw_ratios.tolist()                    # Convert back to list

    # Define 'vw_ratios_desc' (Value-Weight Ratios in descending order)
    vw_ratios_desc = sorted(vw_ratios, reverse=True)  

    # Define empty lists for for loop
    selected_indices = []
    selected_weights = []
    selected_values = []

    # For loop
    for vw_ratio in vw_ratios_desc:
        index = vw_ratios.index(vw_ratio)                  # define product index
    
        if sum(selected_weights) + weights[index] <= 850:
            selected_indices.append(index)                 # get selected products' indices
            selected_weights.append(weights[index])        # get selected products' weights
            selected_values.append(values[index])          # get selected products' values

        else:
            continue
    
    # Print the results (optional)
    print(f"For {i}th warehouse, selected products are: \n{selected_indices} \n")
    print(f"For {i}th warehouse, selected products' weights are: \n{selected_weights} \n")
    print(f"For {i}th warehouse, selected products' values are: \n{selected_values} \n")
    
    print(f"For {i}th warehouse, The Total Weight is: {sum(selected_weights)}")
    print(f"For {i}th warehouse, The Total Value is: {sum(selected_values)}")
    
    # Return [The Total Weight, The Total Value, None]
    
    # 'None' here is for later use (a place for value-per-weight in Problem2)
    return [sum(selected_weights), sum(selected_values), None]  

In [ ]:
# Check the code above
get_max_value(products, 0)
# Make sure you have calculated the total Value and accumulated Weight for One Warehouse
# Again, 'None' here is for later use (a place for value-per-weight in Problem2)

### Step 2: Estimate the Total Value for 300 Warehouses
Let's apply the same calculation process of step 1 for one warehouse to the 299 warehouses left. 

Keep in mind that calculating the estimated total value and accumulated weight of 300 warehouses will be helpful for the later decision of cross-warehouse-transshipment solution in problem 2. 

In [ ]:
# Define 'warehouse_value' as dictionary
warehouse_value = {}

# Fill 'warehouse_value' (itreate 0 ~ 299, instead of 0 ~ 899 by 3)
for i in range(300):
    #write your codes here
    warehouse_value[i] = get_max_value(products, i)

In [ ]:
# Check the code above
print(warehouse_value)         
# Print the warehouse_value generated

## Problem 2: Top Alternative Selections

In this problem, you will simulate the cross-warehouse-transshipment solutions when a random warehouse is out-of-stock during the promotion period.

### Step 1: Generate the 300*300 distance matrix and write the results to "Distances.csv"
Let's generate a distance matrix among the 300 warehouses first. Each of the distances can be generated by using a normal distribution with a mean of 600 and a standard deviation of 400. 

Be careful, all the distances generated should be positive numbers and should be rounded to integers using the round() function. After successfully generating the distance matrix, please write it to a new CSV file called “Distances.csv”.

In [6]:
# Generate the Distance matrix 300*300

#write your codes here
def gen_matrix(matrix_size, random_seed):  # matrix_size here is a height(=or a width) of a square matrix
    
    # Generate distance_numbers randomly
    number_of_distances = sum([i for i in range(matrix_size)])  # the number of distances needed to fill the given sized matrix
    np.random.seed(random_seed)  # use the input of random_seed 
    distance_numbers = np.random.normal(500, 300, (number_of_distances))  # generate random numbers with the given condition
    distance_numbers = [round(np.abs(i)) for i in distance_numbers]  # convert the above to positive integers
    
    # Generate 'matrix_1' filled with zeros
    matrix_1 = np.zeros((matrix_size, matrix_size), dtype = int)
    
    # Fill the upper right corner of 'matrix_1'
    distance_index = 0
        
    for i in range(matrix_size):
        for j in range(matrix_size):
            if j > i:
                matrix_1[i][j] = distance_numbers[distance_index]
                distance_index += 1
    
    # Generate 'matrix_2' by transposing 'matrix_1'
    matrix_2 = matrix_1.T
    
    # Generate 'final_matrix' with the two matrices
    matrix_final = matrix_1 + matrix_2
    
    # Return 'final_matrix'
    return matrix_final    

Now we are going to save the file in 'Distance.csv'

In [ ]:
# Check the code above
distances = gen_matrix(300, 0)
distances  
# Show the distance matrix you generated 
# Check your maxtrix and see if all numbers are positive and rounded to integer

In [8]:
f = open("Distances.csv", "w") # We will create this file.

#write your codes here
for i in distances:
    for j in i:
        f.write(f"{str(j)},")
    f.write("\n")

f.close()

### Step2：Select the Alternative Warehouse ("Helper")

Marlon randomly selects No.p warehouse that is out-of-stock in this simulation.

In [ ]:
'''p is offered by Malron. Do not change the code below.'''

p = np.random.randint(0,300) 
p

There are 3 things to do:
1. Based on the generated p, find the corresponding distance between this warehouse No.p and each of the other Helpers from the distance matrix you generated in Step 1 of Problem 2.
2. Find the corresponding total value and total weight of all products stored in each Helper (you have calculated these numbers in Problem 1).
3. Calculate the “Value_per_Weight” ratio for each Helper, sort the ratio in descending order and choose top helpers with 10 highest “Value_per_Weight”ratios.

#### <center> $Value\_per\_Weight = \frac{Total\_Value}{Total\_Weight}-Distance\times\ Transpotation\_Cost$

In [10]:
#write your codes here

# Update the last place of 'warehouse_value' (that we left as 'None') with value-per-weight between p and i
for i in range(300):
    
    if i != p:
        i_distance = distances[p][i]
        i_total_weight = warehouse_value[i][0]
        i_total_value = warehouse_value[i][1]
        i_value_per_weight = (i_total_value / i_total_weight) - i_distance * 0.015
        
        # Update the last place with value-per-weight between p and i 
        warehouse_value[i][-1] = i_value_per_weight  
        
    else:
        # Update the last place with a very small number to prevent a warehouse from showing itself as a helper
        warehouse_value[i][-1] = (-float('inf'))  

# Flip keys, values of 'warehouse_value'
warehouse_value_flipped = {value[-1]: key for key, value in warehouse_value.items()}

# Find the top 10 helpers
helpers_sorted = sorted(warehouse_value_flipped, reverse=True)

# Check if 'helpers_sorted' has any duplicate values
if len(helpers_sorted) == len(set(helpers_sorted)):
    
    # Final output
    top_10_helpers = []     
    for i in helpers_sorted[:10]:
        top_10_helpers.append((warehouse_value_flipped[i], i))
else:
    print("Warning: Duplicate values in value-per-weight")

In [ ]:
# Check the code above
top_10_helpers
# Show the cross-warehouse-transshipment solution when No.p warehouse is out-of-stock during the promotion period. 
# The solution should contain Helper indexes and the corresponding "Value_Per_Weight" ratios. 

#### Note:

Due to random assignment of No.p warehouse, it's acceptable if your recommended solution changes each time you re-run the code. 

### Submision Requirement:

Please submit all files (Distances.csv, and Fall23_Midterm_Amazon.ipynb files) as one zip file on Canvas. Please add IDs on the name of zip file. For example, 14325_34672_12345.zip.